In [1]:
import sys
import platform
import locale
import dbDictUpload as db
import elasticsearch
from elasticsearch import Elasticsearch, RequestsHttpConnection
import certifi
from elasticsearch import helpers
from keysLegislation import elastic_keys
import insta_scraper as insta
from insta_listkeys import keys
import insta_dictionaries as idict
from InstagramAPI import InstagramAPI
from tags import tag_posts
import insta_s3 as is3
import pprint
pp = pprint.PrettyPrinter(indent=4)

import logging
from datetime import datetime

logger = logging.getLogger(__name__)

In [2]:
def location_geopoint(d):
    if "location_lat" in d.keys():
        logger.info("Location: {}".format(d["url"]))
        d["pin"] = {
            "location" : {
                "lat" : d["location_lat"],
                "lon" : d["location_lng"]
                }
            }

    return d

In [4]:
def main():
    logger.info('Init dicts and database connections...')
    path = ''
    es = Elasticsearch(elastic_keys['endpoint'],
                          http_auth=(elastic_keys['usr'], elastic_keys['pwd']),
                          timeout=30, max_retries=10,retry_on_timeout=True)
    
    #API = InstagramAPI(keys[0]['user_name'],keys[0]['pwd'])
    API = InstagramAPI("****","****")
    API.login()
    for topic,tags in tag_posts.items():
        for tag in tags:
            try:
                logger.info('Tag: {}. Getting media...'.format(tag))
                posts_story = insta.get_tag_feed(tag, API,type_ = 'story')
                logger.info('{} posts'.format(len(posts_story)))
                count_uploded_to_elastic, i = 0, 1
                for media in posts_story:
                    pp.pprint(media)
                    logger.info('{}/{}'.format(i,len(posts_story)))
                    i += 1
                    logger.info('Building entities...')
                    d = idict.media_story_dict(media)
                    d = location_geopoint(d)
                    d['topic'] = topic
                    d['source'] = 'story'
                    #save story in s3
                    logger.info('Saving in s3')
                    bucket = "insta-stories"
                    s3dir = 'stories-photos'
                    print('='*100)
                    #print(d['url'])
                    print(s3dir,d['insta_id2'],path)
                    print('='*100)
                    is3.save_story(d['url'],s3dir,d['insta_id2'],path+"/insta_scripts")
                    s3link = "https://s3-us-west-2.amazonaws.com/{}/{}/{}.{}".format(bucket,s3dir,d['insta_id2'],"png")
                    d['s3_photo'] = s3link
                    if 'video_url' in d.keys():
                        s3dir = 'stories-videos'
                        is3.save_story(d['video_url'],s3dir,d['insta_id2'],path+"/insta_scripts")
                        s3link = "https://s3-us-west-2.amazonaws.com/{}/{}/{}.{}".format(bucket,s3dir,d['insta_id2'],"mp4")
                        d['s3_video'] = s3link
                    #upload to elastic
                    logger.info('Uploading...')
                    d['timestamp'] = datetime.now()
                    print('tattoos','instagram-locations',d,str(d['insta_id2']))
                    #uploaded_elastic = db.single_upload_elastic(es,'tattoos','instagram-locations',d,id_=str(d['insta_id2']))
                    #if uploaded_elastic:
                    #    count_uploded_to_elastic += 1
                #logger.info('{}/{} posts uploaded to elastic for {}'.format(count_uploded_to_elastic,len(posts_story),tag))
                print("Ah?")
            except:
                #print("Ah?")
                logging.exception('CRITICAL ERROR uploading posts for: {}'.format(tag,str(sys.exc_info()[0])))
    API.logout()
    logger.info("Finished!!")

In [5]:
if __name__ == "__main__":
    format_t = "%(asctime)s [%(levelname)s] %(message)s"
    logging.basicConfig(format=format_t,level=logging.INFO)#filename='congressesMain.log')
    main()

2019-02-12 13:49:50,318 [INFO] Init dicts and database connections...
2019-02-12 13:49:50,431 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-12 13:49:50,650 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-12 13:49:51,494 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthe

Request return 429 error!
{'status': 'fail', 'message': 'Please wait a few minutes before you try again.'}


2019-02-12 13:49:53,577 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



Request return 404 error!


2019-02-12 13:49:53,966 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-12 13:49:54,344 [INFO] Tag: tattoos. Getting media...
2019-02-12 13:49:54,430 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



...

2019-02-12 13:49:59,030 [ERROR] CRITICAL ERROR uploading posts for: tattoo
Traceback (most recent call last):
  File "<ipython-input-4-c38f06609b9b>", line 35, in main
    is3.save_story(d['url'],s3dir,d['insta_id2'],path+"/insta_scripts")
  File "/home/ubuntu/Documents/repos/social_networks/instagram_stories/insta_s3.py", line 25, in save_story
    file_name = save_local(insta_id,url,type_,path)
  File "/home/ubuntu/Documents/repos/social_networks/instagram_stories/insta_s3.py", line 11, in save_local
    with open(file_name, 'wb') as out_file:
FileNotFoundError: [Errno 2] No such file or directory: '/insta_scriptsstories-photos/1977862811543456126_5771545632.png'
2019-02-12 13:49:59,034 [INFO] Tag: tatuajes. Getting media...
2019-02-12 13:49:59,156 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.

{   'can_reply': True,
    'can_reshare': True,
    'can_viewer_save': True,
    'caption': None,
    'caption_is_edited': False,
    'caption_position': 0.0,
    'client_cache_key': 'MTk3NzY4MjY2Mzg4ODE5MjUzMQ==.2',
    'code': 'BtyI-eTgQQT',
    'device_timestamp': 15100126099006,
    'expiring_at': 1550064538,
    'filter_type': 0,
    'has_audio': True,
    'has_shared_to_fb': 0,
    'id': '1977682663888192531_1687493915',
    'image_versions2': {   'candidates': [   {   'height': 1138,
                                                 'url': 'https://instagram.fmex6-1.fna.fbcdn.net/vp/ee5ab2dfb3b392cf738bb7e0881a41e6/5C6581AF/t51.12442-15/e35/51612923_285372122134737_8172145042787871817_n.jpg?_nc_ht=instagram.fmex6-1.fna.fbcdn.net',
                                                 'width': 640},
                                             {   'height': 426,
                                                 'url': 'https://instagram.fmex6-1.fna.fbcdn.net/vp/f09c828981212f8abeb292823

2019-02-12 13:50:01,717 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-12 13:50:04,024 [INFO] 15 posts
2019-02-12 13:50:04,041 [INFO] 1/15
2019-02-12 13:50:04,043 [INFO] Building entities...
2019-02-12 13:50:04,044 [INFO] Location: https://instagram.fmex6-1.fna.fbcdn.net/vp/af68eae40a998e9f97c9a50c59afef3c/5C65977D/t51.12442-15/e35/50636922_2272255912993132_4961480093537146055_n.jpg?_nc_ht=instagram.fmex6-1.fna.fbcdn.net
2019-02-12 13:50:04,045 [INFO] Saving in s3
2019-02-12 13:50:04,095 [ERROR] CRITICAL ERROR uploading posts for: tatuaje
Traceback (most recent call last):
  File "<ipython-input-4-c38f06609b9b>", line 35, in main
    is3.save_story(d['url'],s3dir,d['insta_id2'],path+"/insta_scripts")
  File "/home/ubu

{   'can_reply': True,
    'can_reshare': True,
    'can_viewer_save': True,
    'caption': None,
    'caption_is_edited': False,
    'caption_position': 0.0,
    'client_cache_key': 'MTk3NzIwNDE2Mjg0OTc4NTczOA==.2',
    'code': 'BtwcLXfFceK',
    'device_timestamp': 3209028791465428,
    'expiring_at': 1550007511,
    'filter_type': 0,
    'has_audio': False,
    'has_shared_to_fb': 0,
    'id': '1977204162849785738_3000960957',
    'image_versions2': {   'candidates': [   {   'height': 1280,
                                                 'url': 'https://instagram.fmex6-1.fna.fbcdn.net/vp/af68eae40a998e9f97c9a50c59afef3c/5C65977D/t51.12442-15/e35/50636922_2272255912993132_4961480093537146055_n.jpg?_nc_ht=instagram.fmex6-1.fna.fbcdn.net',
                                                 'width': 720},
                                             {   'height': 426,
                                                 'url': 'https://instagram.fmex6-1.fna.fbcdn.net/vp/81b3b9574e137b76a1863

2019-02-12 13:50:04,465 [INFO] Finished!!


In [12]:
API = InstagramAPI(keys[0]['user_name'],keys[0]['pwd'])
#API = InstagramAPI("siddhartasc",'Inst.924406')

API.login()

2019-02-11 19:07:21,582 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-11 19:07:21,766 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



Request return 400 error!
{'status': 'fail', 'error_type': 'checkpoint_challenge_required', 'challenge': {'url': 'https://i.instagram.com/challenge/4047923872/hPoEbVMDA2/', 'lock': True, 'api_path': '/challenge/4047923872/hPoEbVMDA2/', 'hide_webview_header': True, 'native_flow': True, 'logout': False}, 'message': 'challenge_required'}
